# LLMs and Chat Models

In [ ]:
# text-davinci-003
from langchain.llms.openai import OpenAI
# gpt-r.5-turbo
from langchain.chat_models import ChatOpenAI
llm = OpenAI()
chat = ChatOpenAI()

# predict just one string, message
a = llm.predict("How many planets are there")
b = chat.predict("How many planets are there")
a,b

# Predict Messages

In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.1)

In [ ]:
# predict list of messages
# message constructors
# 맥락 및 조건 설정 가능, 작성한 것들 memory에 추가, 
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content = "You are a georgraphy expoert. And you only reply in Inalian.",
    ),
    AIMessage(content= "Ciao, mi chiamo Paolo"),
    HumanMessage(
        content = "What is the distance between Mexico and Thailand. Also, what isyour name?",
    ),
]
chat.predict_messages(messages)

# Prompt Template

In [ ]:
# 템플릿은 일종의 validation 기능을 해서 쓸대없는 문장을 소비하지 않게 만들어줌
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}."
)
prompt = template.format(country_a = "Mexico",country_b = "Thailand")
chat.predict(prompt)


In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        ('system',"You are a georgraphy expoert. And you only reply in {language}."),
        ("ai","Ciao, mi chiamo {name}"),
        ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)
prompt = template.format_messages(
    language = 'Greek',
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand"

)
chat.predict_messages(prompt)

# Output Parser and LangChainExpressionLanguage(LCEL)

In [ ]:
# LangChain Expression Language
from langchain.schema import BaseOutputParser

# LLM 응답을 변형해야 할 때 사용하기 위해서
class CommaOutputParser(BaseOutputParser):
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
p = CommaOutputParser()
p.parse("Hello,how, are, you")
    

In [ ]:
from langchain.prompts import ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ('system', "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else"),
    ("human","{question}")
]
)
prompt = template.format_messages(max_items = 10,question = "What are the planets?")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

In [ ]:
prompt = template.format_messages(max_items = 10, question = "What are the colors")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

- 체인을 이용하면 각각의 predict 결과를 연결해서 사용할 수 있음
- R의 chain 같은 느낌
```
chain1 = template1 | chat | outputparser1
chain2 = template2 | chat | outputparser2
chain_sum = chain1 | chain2 | outputparser_sum
```

In [ ]:
# 코드가 너무 김
# 기존 작업을 위해 작성한 코드: Chat model, 파서 생성, 템플릿 생성, message format, predict, 파서 호출, 파싱 
template = ChatPromptTemplate.from_messages([
    ('system', "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else"),
    ("human","{question}")
]
)
# chain 사용
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the poketmons?"
})

# Chaining Chains
- [invoke 설명](https://python.langchain.com/docs/expression_language/interface)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
# Streaming은 모델의 응답이 생성되는걸 보게 해줌, 
chat = ChatOpenAI(
    temperature=0.1,streaming=True,callbacks=[StreamingStdOutCallbackHandler()]
    )


chef_template = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food")
])

chef_chain = chef_template | chat 

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetereian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    "human","{recipe}"
])
veg_chain = veg_chef_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain


final_chain.invoke({
    "cuisine":"indian"
})


# [Module format](https://python.langchain.com/docs/modules/)
- Model I/O
    - Model Input & Output
    - prompts, language model, output parser


- Retrieval
    - 외부데이터를 모델에 어떻게 적용하느냐

- Chains

- Memory

- Agents


# Fewshot Prompt Template
- prompt template을 디스크에 저장하고 로드할 수 있기 때문에 필요함

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

t = PromptTemplate(
    template = "What is the capital of {country}",
    input_variables=["country"],
)
t.format(country = "France")

t= PromptTemplate.from_template("What is the capital of {country}")
t.format(country = "France")


In [ ]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

example_prompt =  PromptTemplate.from_template("Human:{question}\nAI:{answer}")

# suffix 는 사용자의 질문
prompt = FewShotPromptTemplate(
    example_prompt= example_prompt,
    examples = examples,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
    )


# chat.predict(prompt.format(country = 'Germany'))
chain = prompt | chat
chain.invoke({
    "country": "Turkey"
})

# Few Shot Chat Message Prompt Template

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example prompt는 example과 key값이 일치해야함
example_prompt =  ChatPromptTemplate.from_messages(
    [
    ("human", "What do you know about {country}?"),
    ("ai","{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples = examples,
    )

final_prompt = ChatPromptTemplate.from_messages(
    [
    ("system"," You are a geography expert, you gives short answers"),
    example_prompt,
    ("human","What do you know about {country}?"),
    ],
)

chain = final_prompt | chat
chain.invoke({
    "country": "THailand"
})

In [ ]:
print(final_prompt.format(country = "Germany"))

# Length Based Example Selector


In [88]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


        


chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self,examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

    
example_prompt =  PromptTemplate.from_template("Human:{question}\nAI:{answer}")

# example_selector = LengthBasedExampleSelector(
#     examples = examples,
#     example_prompt = example_prompt,
#     max_length = 80)

example_selector = RandomExampleSelector(
    examples = examples,
)

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt,
    example_selector= example_selector,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
    )

prompt.format(country='Brazil')

'Human:What do you know about Greece?\nAI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# Serialize
- 디스크에서 프롬프트를 가져오는 법
- 이를 통해 prompt를 별도로 저장 가능함


In [2]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

prompt.format(country = "Germany")


'What is the capital of Germany'

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate
chat = ChatOpenAI(
    temperature = 0.1,
    streaming= True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)


prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]


full_prompt = PipelinePromptTemplate(
    final_prompt = final,
    pipeline_prompts= prompts
)

full_prompt.format(
    character = "Pirate",
    example_question = "What is your location",
    example_answer = "Arggg! That is a secret! Arg arg!!",
    question = "What is your fav food?"
)

chain  =  full_prompt | chat
chain.invoke({
    "character" : "Pirate",
    "example_question" : "What is your location",
    "example_answer" : "Arggg! That is a secret! Arg arg!!",
    "question" : "What is your fav food?"
})

Arrr, me favorite food be a good ol' plate o' fish 'n' chips! The crispy batter and tender fish be a delight to me taste buds, matey! Arggg!

AIMessageChunk(content="Arrr, me favorite food be a good ol' plate o' fish 'n' chips! The crispy batter and tender fish be a delight to me taste buds, matey! Arggg!")

# Caching
- LM의 응답을 저장할 수 있음

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache())
# 자동으로 db를 만들며 저장함 third party를 통해 다양한 방법 가능
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True) # 세부 정보 다 볼 수 있음

chat = ChatOpenAI(
    temperature = 0.1,
    # streaming= True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ]
)
chat.predict("How do you make Italian pasta?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [11.03s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients and steps:\n\nIngredients:\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nSteps:\n1. On a clean surface or in a large mixing bowl, create a well with the flour.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it 

"To make Italian pasta, you will need the following ingredients and steps:\n\nIngredients:\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nSteps:\n1. On a clean surface or in a large mixing bowl, create a well with the flour.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n6. After resting, divide the dough into smaller portions for easier handling.\n7. Take one portion and flatten it with your hands or a rolling p

In [21]:
# 속도가 달라지는 것을 볼 수 있음
chat.predict("How do you make Italian pasta?")


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients and steps:\n\nIngredients:\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nSteps:\n1. On a clean surface or in a large mixing bowl, create a well with the flour.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest f

"To make Italian pasta, you will need the following ingredients and steps:\n\nIngredients:\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nSteps:\n1. On a clean surface or in a large mixing bowl, create a well with the flour.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n6. After resting, divide the dough into smaller portions for easier handling.\n7. Take one portion and flatten it with your hands or a rolling p

# Serialize and Calculate Cost

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
chat = ChatOpenAI(
    temperature= 0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("whar is the recipe for soju")
    b = chat.predict("whar is the recipe for bread")
    print(a,b,"\n")
    print(usage)





Here is a simple recipe for making soju at home:

Ingredients:
- 1 cup of rice
- 1 cup of nuruk (Korean fermentation starter)
- 10 cups of water
- 1 tablespoon of yeast (optional, for faster fermentation)
- Sugar (optional, for sweetness)

Instructions:
1. Rinse the rice thoroughly until the water runs clear.
2. Cook the rice in a rice cooker or on the stovetop until it is fully cooked and slightly sticky.
3. Let the cooked rice cool down to room temperature.
4. In a large container or pot, mix the cooled rice, nuruk, and water together.
5. Cover the container with a clean cloth or plastic wrap and let it sit at room temperature for about 7-10 days. Stir the mixture once a day.
6. After the fermentation period, strain the mixture through a cheesecloth or fine mesh strainer to remove any solids.
7. If desired, add sugar to taste and stir until dissolved.
8. Transfer the strained liquid into bottles or jars and seal them tightly.
9. Let the soju age for at least a week in the refrigerato

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm
chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model = "gpt-3.5-turbo"
)
# 특별하게 세팅한 모델 저장 & 호출 가능
chat.save("model.json")

call_chat = load_llm("model.json")
call_chat

/Users/andonghyeon/Desktop/side_project/FULLSTACK-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/andonghyeon/Desktop/side_project/FULLSTACK-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})

# Memory
- 챗봇의 기본
- openai api는 메모리 기능을 포함하지 않음

### Conversation Buffer Memory
- 이전 모든 대화를 저장 -> 대화 내용이 길어질수록 저장
- text completion에 사용함
- 5 종류의 메모리가 있는데 이는 같은 API를 공유함 (save_context,load_memory_variables)
- return_messages 는 chat을 사용할 때 사용

In [8]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"inpuy":"Hi"},{"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'), AIMessage(content='How are you?')]}

In [9]:
memory.save_context({"inpuy":"Hi"},{"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi'),
  AIMessage(content='How are you?')]}

### Conversation Buffer Window Memory
- 대화의 특정 부분만 저장하는 방법 (특정 범위)
- 최근 대화만 집중한다는 단점

In [29]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(
    return_messages = True,
    k=4
)

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})
add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)
memory.load_memory_variables({})


{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [30]:
add_message(5,5)
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

### Conversation Summary Memory
- llm을 사용하는 memory
- conversation을 요약해서 저장
 

In [32]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=llm)

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})


In [33]:
add_message("Hi, my name is Donghyun Ann, I live in South Korea.","Wow that is so cool")
add_message("South Korea is so pretty.","I wish I could go!")


In [34]:
get_history()

{'history': 'The human introduces themselves as Donghyun Ann from South Korea. The AI finds this information cool and expresses a desire to visit South Korea because it is pretty.'}

### Conversation Summary Buffer Memory
- 메모리에 보내온 메시지의 수를 저장
- limit에 도달하면 오래된 메시지는 요약해서 저장


In [48]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit = 100,
    return_messages= True,
    )

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi, my name is Donghyun Ann, I live in South Korea.","Wow that is so cool")
add_message("South Korea is so pretty.","I wish I could go!")

In [49]:
get_history()


{'history': [HumanMessage(content='Hi, my name is Donghyun Ann, I live in South Korea.'),
  AIMessage(content='Wow that is so cool'),
  HumanMessage(content='South Korea is so pretty.'),
  AIMessage(content='I wish I could go!')]}

In [50]:
add_message("How far is Korea form Argentina.","I don't know! Super far!")


In [51]:
get_history()


{'history': [HumanMessage(content='Hi, my name is Donghyun Ann, I live in South Korea.'),
  AIMessage(content='Wow that is so cool'),
  HumanMessage(content='South Korea is so pretty.'),
  AIMessage(content='I wish I could go!'),
  HumanMessage(content='How far is Korea form Argentina.'),
  AIMessage(content="I don't know! Super far!")]}

### Conversation Kknowledge Graph Memory
- Knowledge Graph를 만들어냄 = 가장 중요한 내용만 뽑아낸 요약본

In [65]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationKGMemory(
    llm=llm,
    return_messages= True,
    )

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})

# def get_history():
#     return memory.load_memory_variables({})

add_message("Hi, my name is Donghyun. I live in South Korea.","Wow that is so cool")
# add_message("South Korea is so pretty.","I wish I could go!")

In [68]:
memory.load_memory_variables({"input":"Who is Donghyun?"})

{'history': [SystemMessage(content='On Donghyun: Donghyun lives in South Korea. Donghyun likes Kimchi.')]}

In [70]:
add_message("Donghyun likes Kimchi","Wow that is so cool")
memory.load_memory_variables({"input":"Who is Donghyun?"})


{'history': [SystemMessage(content='On Donghyun: Donghyun lives in South Korea. Donghyun likes Kimchi.')]}

## Memory on LLMChain
- template 안에 memory를 집어넣는 법

In [83]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="chat_history"
)

template = """

    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}

"""

chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt = PromptTemplate.from_template(template),
    verbose = True
)

chain.predict(question = "My name is Danny")



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI talking to a human.

    
    Human:My name is Danny



> Finished chain.


"AI: Hello Danny! It's nice to meet you. How can I assist you today?"

In [84]:
chain.predict(question = "I live in Seongnam")



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI talking to a human.

    Human: My name is Danny
AI: AI: Hello Danny! It's nice to meet you. How can I assist you today?
    Human:I live in Seongnam



> Finished chain.


"AI: That's great, Danny! Seongnam is a beautiful city. Is there anything specific you would like to know or discuss about Seongnam?"

In [85]:
chain.predict(question = "What is my name?")




> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI talking to a human.

    System: The human introduces themselves as Danny and mentions that they live in Seongnam. The AI greets Danny and asks how it can assist him today.
AI: AI: That's great, Danny! Seongnam is a beautiful city. Is there anything specific you would like to know or discuss about Seongnam?
    Human:What is my name?



> Finished chain.


'AI: Your name is Danny. You mentioned it when you introduced yourself earlier. Is there anything else I can assist you with, Danny?'

### Chat Based Memory

In [88]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate,MessagesPlaceholder
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="chat_history",
    return_messages= True
)


prompt = ChatPromptTemplate.from_messages(
    [
    ("system","You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}"),
    ]
)

chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt = prompt,
    verbose = True
)

chain.predict(question = "My name is Danny")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Danny

> Finished chain.


'Hello Danny! How can I assist you today?'

In [89]:
chain.predict(question = "I live in Seongnam")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Danny
AI: Hello Danny! How can I assist you today?
Human: I live in Seongnam

> Finished chain.


"That's great, Danny! Seongnam is a city located in South Korea, known for its vibrant culture and modern amenities. How can I help you with anything related to Seongnam or any other topic you have in mind?"

In [90]:
chain.predict(question = "What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
System: The human introduces themselves as Danny and mentions that they live in Seongnam. The AI responds by acknowledging Danny and providing information about Seongnam. The AI also offers assistance with any topic Danny has in mind.
Human: What is my name?

> Finished chain.


'Your name is Danny.'

### LCEL Based Memory

In [102]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate,MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="history", #default
    return_messages= True
)

prompt = ChatPromptTemplate.from_messages(
    [
    ("system","You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}"),
    ]
)

def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history = load_memory) | prompt | llm


def invoke_chain(question):
    result= chain.invoke(
        {
        "question": question
        }
    )
    memory.save_context(
        {"input":question},
        {"output":result.content},
        )
    print(result)

In [112]:
invoke_chain("My name is Donghyun")

{'question': 'My name is Donghyun'}
content='Hello Donghyun! How can I assist you today? Is there anything specific you would like to know or discuss?'


In [113]:
invoke_chain("What is my name?")


{'question': 'What is my name?'}
content='Your name is Donghyun. You mentioned it earlier in our conversation. How can I assist you, Donghyun?'


# RAG

## [Retrieval](https://python.langchain.com/docs/modules/data_connection/)
- Loader: 다양한 로더가 있고 3rd party 도 있음 -> slack ,telegram, twitter 등
- Transform: 문서 분할
    - 임베딩, 저장을 위해 필요한 파일의 부분들을 전달해줘야 함
    - Text Splitter (각 LLM 마다 텍스트 길이와 토큰 계산해주는게 다르기 때문에 맞춰 써주는게 좋음)


In [120]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader

loader = TextLoader("./files/chapter_one.txt")

loader.load()

[Document(page_content="Part 1, Chapter 1\n\nPart One\n\n\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had

In [123]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./files/chapter_one.pdf")

loader.load()

[Document(page_content="chapter_one.md 2024-01-01\n1 / 9Part 1, Chapter 1\nPart One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into\nhis breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions,\nthough not quickly enough to prevent a swirl of gritty dust from entering along with him.\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for\nindoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide:\nthe face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features.\nWinston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working,\nand at present the electric current was cut off during daylight hours. It was part of the economy drive in\npreparation for Hate Week. The flat was seven flights up, and Winsto

In [132]:
# 파워포인트 html pdf 이미지 등등 다 가능함
from langchain.document_loaders import UnstructuredFileLoader

# 에러가 날 경우  설치가 필요함
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

loader = UnstructuredFileLoader("./files/chapter_one.docx")
loader.load()

[Document(page_content="Part 1, Chapter 1\n\nPart One\n\n\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had

In [134]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter()
loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load()
splitter.split_documents(docs)

[Document(page_content="Part 1, Chapter 1\n\nPart One\n\n\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had

In [135]:
loader.load_and_split(text_splitter=splitter)

[Document(page_content="Part 1, Chapter 1\n\nPart One\n\n\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had